In [1]:
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings 
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI 
from langchain.chains import ConversationalRetrievalChain


In [4]:
file_paths = [
    "/Users/linchihhsien/Documents/Arduino/爬蟲/langchain-practice/Courses_data.csv",
    "/Users/linchihhsien/Documents/Arduino/爬蟲/langchain-practice/CourseScore.csv",
    "/Users/linchihhsien/Documents/Arduino/爬蟲/langchain-practice/Syllabus_Data.csv"
]

all_documents = []

for file_path in file_paths:
    loader = CSVLoader(file_path=file_path)
    documents = loader.load()
    all_documents.extend(documents)  

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(all_documents)

embeddings_model = OpenAIEmbeddings(openai_api_key='input key')

vectorstore = FAISS.from_documents(all_documents, embeddings_model)

vectorstore.save_local("faiss_index")



new_db = FAISS.load_local("faiss_index", embeddings_model, allow_dangerous_deserialization=True)


In [6]:
llm_model = ChatOpenAI(openai_api_key='input key') 

#chain = ConversationalRetrievalChain.from_llm(llm=llm_model, retriever=new_db.as_retriever(search_kwargs={"k": 3, "fetch_k": 10}, search_type="mmr"))
chain = ConversationalRetrievalChain.from_llm(llm=llm_model, retriever=new_db.as_retriever())

query = "請根據資料,列出甜度超過6的課程" 
response = chain.invoke({"question": query, "chat_history": []}) 
print(response)
 


{'question': '請根據資料,列出甜度超過6的課程', 'chat_history': [], 'answer': '甜度超過6的課程有：\n1. A9-304 應用化學與實驗 (甜度: 6)\n2. A9-305 應用化學與實驗 (甜度: 6)\n3. A9-114 應用統計 (甜度: 7)'}


In [ ]:
# 定義你的提示模板
prompt = """
課程背景：管理學院中所有系所所開設課程中，出席情況通常會在一定的比例下影響總分。
問題：工資管系統計軟體與繪圖這門課中出席分數佔比多少？
"""

保險數學這門課中出席分數佔比為10%。
page_content='人工智慧導論\n方法\t百分比%\n作業 Assignments\t30\n期中考 Midterm Exam\t30\n個人書面報告 Projects\t30\n出席 Participation\t10\n工資系Industrial and Information Management\n方法\t百分比%\n講授 Lecture\t70\n討論 Discussion\t20\n報告 Presentation\t10\n進度說明 Progress Description\n1\tenvironment setting\n2\tdata type & operator introduction\n3\tif/else & loop\n4\tlist/tuple introduction\n5\tdict/set introduction\n6\tfunction & try/except\n7\tclass/objects\n8\tregular expression\n9\tmodule introduction– numpy\n10\tmodule introduction - pandas\n11\tmodule introduction - sklearn\n12\tmodule introduction– matplot & seaborn\n13\tKeras-CNN(mnist)\n課程概述 Course Description\n       Driven by advances in computer power, availability of large data, and learning algorithms, Artificial Intelligence (AI) has now achieves the impressive successes and become an important science in academia and industry as well. AI focuses on developing intelligent systems to help people to solve various complicated decision-ma

In [38]:
# 定義你的提示模板
prompt = """The following are excerpts from conversations with an AI
assistant. The assistant is typically concise , producing
accurate responses to the users questions. Here are some
examples: 

User: 工資管系統計軟體與繪圖這門課中出席分數佔比多少？
AI: 工資管系統計軟體與繪圖這門課中出席分數佔比為20%

User: 工資管系計算機該論這門課中出席分數佔比多少？
AI: 因為在我的資料庫中工資管系計算機該論這門課的課程大綱中並沒有描述出席分數的比重因此我無法回答你

User: 生物統計這門課中出席分數佔比多少？
課程背景：生物統計這門課中不包含出席分數
AI: 這門課程中出席分數並不會影響總分

User: 告訴我課程甜度超過8的課程?
AI: 課程甜度為9的是戲劇與電影以及社會電影 課程甜度為10的有3堂


User: 保險數學這門課中出席分數佔比多少？
AI: """

llm_model = ChatOpenAI(openai_api_key='input key') 
chain = ConversationalRetrievalChain.from_llm(llm=llm_model, retriever=new_db.as_retriever())

# 設置查詢和 prompt
query = "課程甜度8的所有課程"
prompt = "請根據資料列出，" + query

# 調用鏈並顯示回應
response = chain.invoke({"question": query, "prompt": prompt, "chat_history": []})
print(response)

{'question': '課程甜度8的所有課程', 'prompt': '請根據資料列出，課程甜度8的所有課程', 'chat_history': [], 'answer': '課程名稱: A9-014 電影藝術與生活\n課程名稱: A9-005 戲劇與電影'}
